In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [39]:
noise = pd.read_csv("noise")
up = pd.read_csv("up")
down = pd.read_csv("down")

In [40]:
noise[['X','Y','Time']] = noise[['X','Y','Time']].diff(periods=21, axis=0)
noise = noise[21:].reindex()

up[['X','Y','Time']] = up[['X','Y','Time']].diff(periods=21, axis=0)
up = up[21:].reindex()

down[['X','Y','Time']] = down[['X','Y','Time']].diff(periods=21, axis=0)
down = down[21:].reindex()

In [41]:
print(noise.isna().mean().mean() == 0)
print(up.isna().mean().mean() == 0)
print(down.isna().mean().mean() == 0)

True
True
True


In [42]:
noise['Vx'] , noise['Vy']= noise['X']/noise['Time'] , noise['Y']/noise['Time']
up['Vx'] , up['Vy']= up['X']/up['Time'] , up['Y']/up['Time']
down['Vx'] , down['Vy']= down['X']/down['Time'] , down['Y']/down['Time']

#noise = noise.drop(['X','Y','Time'],axis = 1)
#up = up.drop(['X','Y','Time'],axis = 1)
#down = down.drop(['X','Y','Time'],axis = 1)

In [43]:
down

,ID,X,Y,Time,Vx,Vy
21,0.0,13.0,-13.0,0.044053,295.101759,-295.101759
22,1.0,8.0,-13.0,0.044053,181.601082,-295.101759
23,2.0,3.0,-17.0,0.044053,68.100406,-385.902300
24,3.0,1.0,-21.0,0.044053,22.700135,-476.702841
25,4.0,-2.0,-26.0,0.044053,-45.400271,-590.203518
...,...,...,...,...,...,...
87775,16.0,-28.0,19.0,0.040041,-699.287925,474.516806
87776,17.0,-19.0,-3.0,0.040041,-474.516806,-74.923706
87777,18.0,-23.0,-7.0,0.040041,-574.415081,-174.821981
87778,19.0,-20.0,0.0,0.040041,-499.491375,0.000000


In [44]:
up.shape[0] + down.shape[0] + noise.shape[0]

197526

In [45]:
n = noise.values.reshape(-1,1,21,6)
u = up.values.reshape(-1,1,21,6)
d = down.values.reshape(-1,1,21,6)


In [49]:
u[0,:,:,:]

array([[[ 0.00000000e+00,  8.00000000e+00, -2.00000000e+00,
          4.49748039e-02,  1.77877374e+02, -4.44693434e+01],
        [ 1.00000000e+00, -1.40000000e+01,  1.10000000e+01,
          4.49748039e-02, -3.11285404e+02,  2.44581389e+02],
        [ 2.00000000e+00, -1.10000000e+01,  5.00000000e+00,
          4.49748039e-02, -2.44581389e+02,  1.11173358e+02],
        [ 3.00000000e+00, -5.00000000e+00, -7.00000000e+00,
          4.49748039e-02, -1.11173358e+02, -1.55642702e+02],
        [ 4.00000000e+00, -1.00000000e+00, -1.30000000e+01,
          4.49748039e-02, -2.22346717e+01, -2.89050732e+02],
        [ 5.00000000e+00, -1.00000000e+00, -1.80000000e+01,
          4.49748039e-02, -2.22346717e+01, -4.00224091e+02],
        [ 6.00000000e+00,  4.00000000e+00, -1.70000000e+01,
          4.49748039e-02,  8.89386868e+01, -3.77989419e+02],
        [ 7.00000000e+00,  5.00000000e+00, -2.10000000e+01,
          4.49748039e-02,  1.11173358e+02, -4.66928106e+02],
        [ 8.00000000e+00,  4.000

In [122]:
#data = pd.concat([up,down,noise], axis=0).reindex()

In [47]:
N, C, H, W = n.shape[0] + u.shape[0] + d.shape[0], n.shape[1],n.shape[2] , n.shape[3] 
N, C, H, W

(9406, 1, 21, 6)

In [136]:
#X = np.concatenate((n, u ,d), axis=0)

In [137]:
#y = torch.randint(0, num_classes, (N,))


In [48]:
# 1) Hyperparameters and synthetic data setup
#N, C, H, W = u.shape#1000, 1, 21,6  # samples, channels, height, width
num_classes = 3
batch_size = 64
epochs = 15
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Synthetic dataset: random "images" and integer labels
X = torch.from_numpy(np.concatenate((n, u ,d), axis=0))
y = torch.from_numpy(np.concatenate((np.zeros(n.shape[0]).reshape(-1,) , np.ones(u.shape[0]).reshape(-1,),(np.ones(d.shape[0])+1).reshape(-1,)), axis=0))


dataset = TensorDataset(X, y)
train_size = int(0.8 * N)
val_size = N - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)

# 2) Define the CNN with an embedding layer
class CNNEmbedder(nn.Module):
    def __init__(self, in_channels=1, embed_dim=128, num_classes= 3):
        super(CNNEmbedder, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        # compute flatten dimension: channels=64, spatial=H/4 x W/4
        self.flatten_dim = 64 * (H // 4) * (W // 4)
        self.embed = nn.Linear(self.flatten_dim, embed_dim)
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        emb = torch.relu(self.embed(x))       # embedding layer
        out = self.classifier(emb)            # logits
        return out, emb

model = CNNEmbedder(in_channels=C, embed_dim = 64, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 3) Training loop
model.train()
for epoch in range(epochs):
    total_loss, total_correct = 0.0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, _ = model(xb.float())
        loss = criterion(logits, yb.long())
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)
        total_correct += (logits.argmax(dim=1) == yb).sum().item()

    avg_loss = total_loss / train_size
    train_acc = total_correct / train_size
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}, Acc: {train_acc:.4f}")



Epoch 1/15 - Loss: 1.2463, Acc: 0.6853
Epoch 2/15 - Loss: 0.4196, Acc: 0.8526
Epoch 3/15 - Loss: 0.3597, Acc: 0.8768
Epoch 4/15 - Loss: 0.3231, Acc: 0.8876
Epoch 5/15 - Loss: 0.2582, Acc: 0.9116
Epoch 6/15 - Loss: 0.2483, Acc: 0.9136
Epoch 7/15 - Loss: 0.2610, Acc: 0.9083
Epoch 8/15 - Loss: 0.2178, Acc: 0.9248
Epoch 9/15 - Loss: 0.2012, Acc: 0.9305
Epoch 10/15 - Loss: 0.2034, Acc: 0.9296
Epoch 11/15 - Loss: 0.1708, Acc: 0.9399
Epoch 12/15 - Loss: 0.1771, Acc: 0.9402
Epoch 13/15 - Loss: 0.1761, Acc: 0.9397
Epoch 14/15 - Loss: 0.1480, Acc: 0.9490
Epoch 15/15 - Loss: 0.1310, Acc: 0.9552


In [144]:
# 4) Extract embeddings for train and validation sets
def extract_embeddings(loader):
    model.eval()
    all_emb, all_labels = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.float().to(device)
            _, emb = model(xb)
            all_emb.append(emb.cpu())
            all_labels.append(yb)

    embeddings = torch.cat(all_emb, dim=0).numpy()
    labels = torch.cat(all_labels, dim=0).numpy()
    return embeddings, labels

X_train_emb, y_train_np = extract_embeddings(train_loader)
X_val_emb, y_val_np     = extract_embeddings(val_loader)

# 5) Train XGBoost on embeddings
xgb = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    objective='multi:softmax',
    num_class=num_classes,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
xgb.fit(X_train_emb, y_train_np)

# 6) Evaluate
y_pred = xgb.predict(X_val_emb)
acc = accuracy_score(y_val_np, y_pred)
print(f"XGBoost on CNN embeddings accuracy: {acc:.4f}")


XGBoost on CNN embeddings accuracy: 0.9373


In [151]:
def predict_from_list(matrix_list):
    # matrix_list: list of HxW lists or arrays
    arr = np.array(matrix_list, dtype=np.float32)  # shape (N, H, W)
    if arr.ndim == 2:
        arr = arr[np.newaxis, ...]
    # add channel dimension
    tensor = torch.from_numpy(arr).unsqueeze(1).to(device)  # shape (N,1,H,W)

    model.eval()
    with torch.no_grad():
        _, emb = model(tensor)
    emb_np = emb.cpu().numpy()  # shape (N, embed_dim)
    preds = xgb.predict(emb_np)
    return preds

In [156]:
matrix_list = u[284,:,:,:]
predict_from_list(matrix_list)

array([1])

In [158]:
torch.save(model.state_dict(), "cnn_embedder_state.pt")
xgb.save_model("xgb_embedder.json")

In [159]:
scripted = torch.jit.script(model.cpu())  
scripted.save("cnn_embedder_ts.pt")
xgb.save_model("xgb_embedder.json")